In [1]:
from progressbar import Bar, ETA, \
    AdaptiveETA, Percentage, \
    ProgressBar 
widgets = [Percentage(),
            ' ', Bar(),
            ' ', ETA(),
            ' ', AdaptiveETA()]
pbar = ProgressBar(widgets=widgets)
from pcapreader import PcapReader
pcapToDf = PcapReader.pcapToDf
import os
import pandas as pd
from tabulate import tabulate
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)




In [2]:
def refineLatexTableStringScientificPaper(latexTableString):
    # Hacks for multiindex dataframe
    multiColFirst = latexTableString.find("\multicolumn")
    latexTableString = latexTableString[:multiColFirst+16] + 'c' + latexTableString[multiColFirst+17:]
    startRemove = latexTableString.find('\multicolumn',multiColFirst+1)
    latexTableString = latexTableString[:startRemove] + latexTableString[startRemove+18:]
    # Adding a horizontal line for multindex table
    latexTableString = latexTableString[:latexTableString.find('\\\\')+len('\\\\')] + '\cline{2-5}' + latexTableString[latexTableString.find('\\\\')+len('\\\\'):]
    # Find caption and put at the bottom
    captionIndex = latexTableString.find("\caption")
    captionEndIndex = latexTableString.find("}",captionIndex+1)+1
    endTableIndex = latexTableString.find("\end{tabular}")
    endTableEndIndex = endTableIndex + len("\end{tabular}")
    latexTableString[:endTableIndex] + latexTableString[endTableEndIndex:]
    latexTableString = latexTableString[:endTableEndIndex] + latexTableString[captionIndex:captionEndIndex] + latexTableString[endTableEndIndex:]
    latexTableString = latexTableString[:captionIndex] + latexTableString[captionEndIndex:]
    return latexTableString

In [3]:
## Get All pcap filenames:
def getFiles(baseDir):
    victimFilenames = []
    attackerFilenames = []
    reflectorFilenames = []
    for root, dirs, files in os.walk(baseDir):
        for file in files:
            if file.endswith(".pcapng"):
                if "victim" in file:
                    victimFilenames.append(os.path.join(root, file))
                if "attacker" in file:
                    attackerFilenames.append(os.path.join(root, file))
                if "reflector" in file:
                    reflectorFilenames.append(os.path.join(root,file))
    return attackerFilenames,victimFilenames,reflectorFilenames

In [4]:
## Get attacker bytes sent
def getAttackBytes(attackerFilenames,Protocol):
    pbar = ProgressBar(widgets=widgets)
    attackerBytes = []
    for attackFile in pbar(attackerFilenames):
        # Get attack level
        start = attackFile.find('level')
        end = start+len('level')+1
        level = attackFile[start:end]
        attackerDf = pcapToDf(attackFile,True)
        # By finding the TFTP packets and summing the byte lengths we get the total number of bytes send by the attacker
        attackerDf = attackerDf.loc[attackerDf['Protocol'].isin([Protocol])]
        attackerBytesSent = attackerDf["Length"].sum()
        attackerBytes.append({'Level':level,'Attacker Outbound':attackerBytesSent})
    attackerBytes = pd.DataFrame(attackerBytes).sort_values('Level')
    return attackerBytes

In [5]:
## Get Victim bytes received
def getVictimBytes(victimFilenames):
    pbar = ProgressBar(widgets=widgets)
    victimBytes = []
    for victimFile in pbar(victimFilenames):
        # Get attack level
        start = victimFile.find('level')
        end = start+len('level')+1
        level = victimFile[start:end]
        victimDf = pcapToDf(victimFile,True)
        # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
        victimBytesReceived = victimDf.loc[(victimDf['Protocol'] == "UDP") & (victimDf['UDP Destination Port'] == 50040)]["Length"].sum()
        victimBytes.append({'Level':level,'Victim Inbound':victimBytesReceived}) 
    victimBytes = pd.DataFrame(victimBytes).sort_values('Level')
    return victimBytes

In [6]:
## Get Reflector bytes received and sent
def getReflectorBytes(reflectorFilenames,Protocol):
    pbar = ProgressBar(widgets=widgets)
    reflectorBytes = []
    for reflectorFile in pbar(reflectorFilenames):
        # Get attack level
        start = reflectorFile.find('level')
        end = start+len('level')+1
        level = reflectorFile[start:end]
        reflectorDf = pcapToDf(reflectorFile,True)
        # In the victim pcap filtering by destination port 50040 (the tftp servers source port) gives the tftp data transfered to the victim
        reflectorBytesReceived = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['UDP Source Port'] == 50040)]["Length"].sum()
        reflectorBytesSent = reflectorDf.loc[(reflectorDf['Protocol'] == Protocol) & (reflectorDf['UDP Destination Port'] == 50040)]["Length"].sum()
        reflectorBytes.append({'Level':level,'Reflector Inbound':reflectorBytesReceived,'Reflector Outbound':reflectorBytesSent})
    reflectorBytes = pd.DataFrame(reflectorBytes).sort_values('Level')
    return reflectorBytes

In [7]:
## Data manipulation for presentation
def getStatDf(attackerBytes,victimBytes,reflectorBytes):
    StatDf = pd.DataFrame(columns=pd.MultiIndex.from_tuples([("Victim", "Inbound"), ("Reflector", "Inbound"), ("Reflector", "Outbound"), ("Attacker", "Outbound")]))
    StatDf["Level"] = victimBytes['Level'].apply(lambda x: x.strip('level'))
    StatDf[("Victim","Inbound")] = victimBytes['Victim Inbound']
    StatDf[("Reflector","Inbound")] = reflectorBytes['Reflector Inbound']
    StatDf[("Reflector","Outbound")] = reflectorBytes['Reflector Outbound']
    StatDf[("Attacker","Outbound")] = attackerBytes['Attacker Outbound']
    StatDf['Amplification factor'] = StatDf[("Victim","Inbound")] / StatDf[("Attacker","Outbound")]
    column_to_move = StatDf.pop("Level")
    StatDf.insert(0, "Level", column_to_move)
    return StatDf

# TFTP DATA OWN TOOL

In [8]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_tftp_own_tool')

In [10]:
# Read pcap files
attackerBytes = getAttackBytes(attackerFilenames,Protocol="TFTP")
victimBytes = getVictimBytes(victimFilenames)
reflectorBytes = getReflectorBytes(reflectorFilenames,Protocol="TFTP")

100% |############################################| Time: 0:02:31 Time: 0:02:31
100% |############################################| Time: 0:00:03 Time: 0:00:03
100% |############################################| Time: 0:00:55 Time: 0:00:55


In [11]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerBytes,victimBytes,reflectorBytes)
latexTableString = StatDf.to_latex(column_format='ccrrrc',index=False,caption="TFTP DDoS test traffic",label="TFTP_Test_Traffic",position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

\begin{table}[H]
\centering

\label{TFTP_Test_Traffic}
\begin{tabular}{ccrrrc}
\toprule
Level &  Victim & \multicolumn{2}{c}{Reflector} &  Attacker & Amplification factor \\\cline{2-5}
      & Inbound &   Inbound & Outbound & {Outbound} \\
\midrule
    0 &   48104 &     11160 &   639096 &       840 &            57.266667 \\
    1 &  508528 &     17820 &   508528 &      8880 &            57.266667 \\
    2 & 1813712 &     80640 &  1813712 &     80640 &            22.491468 \\
    3 & 2701042 &    781080 &  2704888 &    781080 &             3.458086 \\
    4 & 3171468 &  85571024 &  2925288 & 236692080 &             0.013399 \\
    5 & 3096028 &  75819480 &  2696964 & 238204140 &             0.012997 \\
\bottomrule
\end{tabular}\caption{TFTP DDoS test traffic}
\end{table}



# SNMP DATA OWN TOOL

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_snmp_own_tool')

In [ ]:
# Read pcap files
attackerBytes = getAttackBytes(attackerFilenames,Protocol="SNMP")
victimBytes = getAttackBytes(victimFilenames)
reflectorBytes = getAttackBytes(reflectorFilenames,Protocol="SNMP")

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerBytes,victimBytes,reflectorBytes)
latexTableString = StatDf.to_latex(column_format='ccrrrc',index=False,caption="SNMP DDoS test traffic",label="SNMP_Test_Traffic",position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)

# SSDP DATA OWN TOOL

In [ ]:
# Get pcap files
attackerFilenames,victimFilenames,reflectorFilenames = getFiles('./pcap_ssdp_own_tool')

In [ ]:
# Read pcap files
attackerBytes = getAttackBytes(attackerFilenames,Protocol="SSDP")
victimBytes = getAttackBytes(victimFilenames)
reflectorBytes = getAttackBytes(reflectorFilenames,Protocol="SSDP")

In [ ]:
#print(tabulate(StatDf,headers='keys',tablefmt='fancy_grid',showindex=False))
StatDf = getStatDf(attackerBytes,victimBytes,reflectorBytes)
latexTableString = StatDf.to_latex(column_format='ccrrrc',index=False,caption="SSDP DDoS test traffic",label="SNMP_Test_Traffic",position='H') # For getting the table into the report
latexTableString = refineLatexTableStringScientificPaper(latexTableString)
print(latexTableString)